In [1]:
import numpy
import xarray as xr
import igutils

dir(igutils)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

In [2]:
grid = xr.open_dataset(
    "/Users/mbianco/Dev/playground/icongrid/icon_grid_0030_R02B03_G.nc"
)
grid

<xarray.Dataset> Size: 5MB
Dimensions:                         (cell: 5120, nv: 3, vertex: 2562, ne: 6,
                                     edge: 7680, no: 4, nc: 2,
                                     max_stored_decompositions: 4, two_grf: 2,
                                     cell_grf: 14, max_chdom: 1, edge_grf: 24,
                                     vert_grf: 13)
Coordinates:
    clon                            (cell) float64 41kB ...
    clat                            (cell) float64 41kB ...
    vlon                            (vertex) float64 20kB ...
    vlat                            (vertex) float64 20kB ...
    elon                            (edge) float64 61kB ...
    elat                            (edge) float64 61kB ...
Dimensions without coordinates: cell, nv, vertex, ne, edge, no, nc,
                                max_stored_decompositions, two_grf, cell_grf,
                                max_chdom, edge_grf, vert_grf
Data variables: (12/91)
    clon_vertices                   (cell, nv) float64 123kB ...
    clat_vertices                   (cell, nv) float64 123kB ...
    vlon_vertices                   (vertex, ne) float64 123kB ...
    vlat_vertices                   (vertex, ne) float64 123kB ...
    elon_vertices                   (edge, no) float64 246kB ...
    elat_vertices                   (edge, no) float64 246kB ...
    ...                              ...
    edge_dual_normal_cartesian_x    (edge) float64 61kB ...
    edge_dual_normal_cartesian_y    (edge) float64 61kB ...
    edge_dual_normal_cartesian_z    (edge) float64 61kB ...
    cell_circumcenter_cartesian_x   (cell) float64 41kB ...
    cell_circumcenter_cartesian_y   (cell) float64 41kB ...
    cell_circumcenter_cartesian_z   (cell) float64 41kB ...
Attributes: (12/43)
    title:                    ICON grid description
    institution:              Max Planck Institute for Meteorology/Deutscher ...
    source:                   git@git.mpimet.mpg.de:GridGenerator.git
    revision:                 2cdf99bec403902989148ebb4bd38c218a64a1b3
    date:                     20190405 at 141502
    user_name:                Rene Redler (m300083)
    ...                       ...
    topography:               modified SRTM30
    symmetry:                 along equator
    subcentre:                1
    number_of_grid_used:      30
    ICON_grid_file_uri:       http://icon-downloads.mpimet.mpg.de/grids/publi...
    NCO:                      netCDF Operators version 4.7.5 (Homepage = http...

In [ ]:
# def check_consistency(grid):
# 	"""
# 	If this function fails, then it means that some in umbering is not right in the file.
# 	"""
	
# 	for i in grid.vertex.values:
# 		for e in grid.edges_of_vertex.values.T[i]:
# 			if e > 0:
# 				for c in grid.adjacent_cell_of_edge.values.T[e-1]:
# 					for v in grid.vertex_of_cell.values.T[c-1]:
# 						if v-1!=i and not v in grid.vertices_of_vertex.values.T[i,:]:
# 							print(f"Error for vertex i={i}, with edge e={e}, with cell c={c} and vertex v={v} is not in {grid.vertices_of_vertex.values.T[i,:]}")
# 							assert False


AttributeError: module 'igutils' has no attribute 'check_consistency'

In [ ]:
eov = [i for x, i in zip(grid.vertices_of_vertex.values[5,:], range(len(grid.vertices_of_vertex.values[5,:]))) if int(x) <= 0]

In [ ]:
eov

In [ ]:
# if the longitudes of the are repeated for the two halves of the array then the grid is not an icosahedron but a symmetric grid
print(f"grid.latitude_vertices.values[eov] = {grid.latitude_vertices.values[eov]}")
print(f"grid.longitude_vertices.values[eov] = {grid.longitude_vertices.values[eov]}")

In [ ]:
def short_path(v1, v2s, grid):
    print(f"computing {v1} -> {v2s}")
    queue = [v1]
    mark = {v1: 0}
    #i = 0
    while queue:
        u = queue.pop(0)
        #i = i+1
        if u in v2s:
            print(f"{v1} to {u} path is {mark[u]} long")
        for v in grid.vertices_of_vertex.values[:,u-1]:
            if v > 0 and not v in mark:
                mark[v] = mark[u] + 1
                #print(f"{i} -> {u} -> {v} (mark = {mark[u]})         ", end='\r')
                queue.append(v)
    return mark   

In [ ]:
def backtrack(src, dst, mark, grid):
	path1 = [dst+1]
	u = dst+1
	#distance = mark[dst]
	while u != src+1:
		vs = grid.vertices_of_vertex.values[:, u-1]
		#print(f"u = {u}")
		ok = False
		for v in vs:
			if v > 0:
				#print(f"    src = {src}, u = {u}, mark[u] = {mark[int(u)]}, v = {v}, mark[v] = {mark[int(v)]}")
				if mark[v] == mark[u]-1:
					#distance = mark[v]
					ok = True
					path1.insert(0, int(v))
					u = v
					break
		if not ok:
			raise Exception("WHATTT!?!?")
		#print(path1)
	return path1

In [ ]:
import gc
#short_path(f['edge_vertices'][0,eov[0]-1], f['vertices_of_vertex'][0,f['edge_vertices'][1,eov[0]-1]])
paths = []
for i in range(0, len(eov)):
    mark = short_path(eov[i]+1, [x+1 for x in eov], grid)
    paths.append([])
    for n in range(1, len(eov)):
        paths[-1].append(backtrack(eov[i], eov[n], mark, grid))
    del(mark)
    gc.collect()

In [ ]:
interesting_path_length = min([ len(x) for x in paths[0][1:]])
print(f"{interesting_path_length=}")

In [ ]:
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
voc = grid.vertices_of_vertex.values

fig = plt.figure(figsize=(50, 40)) # Need to find a way of setting the size right...
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())
ax.set_global()
ax.add_feature(cfeature.LAND, zorder=0, edgecolor="black")
pad = ''
v = 0
transformatio = ccrs.Geodetic()
for v1, v2, v3, v4, v5, v6 in grid.vertices_of_vertex.values.T - 1:
    i = np.array([v, v1])
    for vi in [v1, v2, v3, v4, v5]:
        i = np.array([v, vi])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c= 'k' if v6<=0 else "k",
            lw=3 if v6<=0 else 1,
            alpha=1 if v6<=0 else 0.1,
            transform=transformatio,
        )
    if v6 > 0:
        i = np.array([v, v6])
        plt.plot(
            np.rad2deg(grid.vertices_of_vertex.vlon[i]),
            np.rad2deg(grid.latitude_vertices.vlat[i]),
            c="k",
            lw=1,
            alpha=0.1,
            transform=transformatio,
        )
    if True: # v6<=0: Put this on to print only the ids of the pentagons (useful if the grid is big)
        plt.text(np.rad2deg(grid.vertices_of_vertex.vlon[v]),
                 np.rad2deg(grid.vertices_of_vertex.vlat[v]),
                 str(v+1),
                 transform=transformatio
                )
    v=v+1
# v = 0
# for v1, v2, v3, v4, v5, v6 in grid.vertices_of_vertex.values.T - 1:
#     i = np.array([v, v1])
#     if v6<=0:
#         print(f"v = {v}")
#         for vi in [v1, v2, v3, v4, v5]:
#             i = np.array([v, vi])
#             plt.plot(
#                 np.rad2deg(grid.vertices_of_vertex.vlon[i]),
#                 np.rad2deg(grid.latitude_vertices.vlat[i]),
#                 c= 'k' if v6<=0 else "k",
#                 lw=3 if v6<=0 else 1,
#                 alpha=1 if v6<=0 else 0.1,
#                 transform=transformatio,
#             )
#         plt.text(np.rad2deg(grid.vertices_of_vertex.vlon[v]),
#                 np.rad2deg(grid.vertices_of_vertex.vlat[v]),
#                 str(v+1),
#                 transform=transformatio
#                 )
#     v=v+1

colors = ['r', 'g', 'y', 'c']
count = 0
for thepaths in paths:
    for path1 in thepaths:
        print(f"[{count}] length of path = {len(path1)} from {path1[0]} [{float(grid.vertices_of_vertex.vlon[path1[0]-1])}, {float(grid.vertices_of_vertex.vlat[path1[0]-1])}] to {path1[-1]} [{float(grid.vertices_of_vertex.vlon[path1[-1]-1])}, {float(grid.vertices_of_vertex.vlat[path1[-1]-1])}]")
        if len(path1) <= interesting_path_length:
            
            for ind in range(len(path1)-1):
                #print(f"from vertex {path1[ind]-1} to {path1[ind+1]-1}")
                i = np.array([path1[ind]-1, path1[ind+1]-1])
                plt.plot(
                    np.rad2deg(grid.vertices_of_vertex.vlon[i]),
                    np.rad2deg(grid.latitude_vertices.vlat[i]),
                    c=colors[count%len(colors)],
                    lw=4,
                    alpha=1,
                    transform=transformatio,
                )
            count = count+1

plt.show()
plt.close()